## 深度学习特征

提取CT、MRI、内镜、Xray等影像数据的深度学习特征。

### Onekey步骤

1. 将待提取的数据转化成jpg，可以参考使用OKT-convert2jpg或者OKT-crop_max_roi两个Onekey工具。
2. 获取到指定目录的所有图像数据。
3. 选择要提取什么样的模型的深度学习特征，目前Onekey支持主流的深度学习模型。（可以考虑使用Onekey进行迁移学习）
4. 提取特征，保存特征文件。

In [1]:
from onekey_algo.custom.Manager import onekey_show
onekey_show('深度学习特征提取')

[2025-12-03 16:34:18 - <frozen onekey_algo.custom.Manager>: 159]	INFO	播放视频功能已经设置成：Disable！


## 获取待提取特征的文件

提供两种批量处理的模式：
1. 目录模式，提取指定目录下的所有jpg文件的特征。
2. 文件模式，待提取的数据存储在文件中，每行一个样本。

当然也可以在最后自己指定手动提取指定若干文件。

In [2]:
#### 获取数据
from onekey_algo.custom.Manager import onekey_show
onekey_show('深度学习特征提取|获取数据')

[2025-12-03 16:34:18 - <frozen onekey_algo.custom.Manager>: 159]	INFO	播放视频功能已经设置成：Disable！


In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
from onekey_algo import get_param_in_cwd

os.makedirs('features', exist_ok=True)
mydir = get_param_in_cwd('data_pattern')
samples = [os.path.join(mydir, f) for f in os.listdir(mydir) if f.endswith('.jpg') or f.endswith('.png')]
samples

['D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\100.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1001.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1003.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\101.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1015.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1024.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1026.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1029.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\103.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1030.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1031.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1033.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1036.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\1037.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\104.nii.png',
 'D:\\\\task1DL\\\\11.3\\\\调参761\\\\ROIonly\\10

## 确定提取特征

通过关键词获取要提取那一层的特征。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| AlexNet      | alexnet                                                      |
| VGG          | vgg11, vgg11_bn, vgg13, vgg13_bn, vgg16, vgg16_bn, vgg19_bn, vgg19 |
| ResNet       | resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2 |
| DenseNet     | densenet121, densenet169, densenet201, densenet161           |
| Inception    | googlenet, inception_v3                                      |
| SqueezeNet   | squeezenet1_0, squeezenet1_1                                 |
| ShuffleNetV2 | shufflenet_v2_x2_0, shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5 |
| MobileNet    | mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small         |
| MNASNet      | mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3              |

In [4]:
#### 获取数据
from onekey_algo.custom.Manager import onekey_show
onekey_show('深度学习特征提取|确定模型和特征')

[2025-12-03 16:34:18 - <frozen onekey_algo.custom.Manager>: 159]	INFO	播放视频功能已经设置成：Disable！


In [5]:
from onekey_algo.custom.components.comp2 import extract, print_feature_hook, reg_hook_on_module, \
    init_from_model, init_from_onekey

model_name = get_param_in_cwd('model_name')
model, transformer, device = init_from_onekey(os.path.join(get_param_in_cwd('model_root'), model_name, 'viz'))
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

[2025-12-03 16:34:19 - <frozen onekey_algo.custom.components.comp2>: 235]	INFO	模型参数：{'pretrained': False, 'model_name': 'resnet50', 'num_classes': 2, 'in_channels': 3}


Feature name:  || Module: ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kerne

## 提取特征

`Feature name:` 之后的名称为要提取的特征名，例如`layer3.0.conv2`, 一般深度学习特征提取最后一层，例如`avgpool`

In [6]:
#### 提取特征
from onekey_algo.custom.Manager import onekey_show
onekey_show('深度学习特征提取|提取特征')

[2025-12-03 16:34:21 - <frozen onekey_algo.custom.Manager>: 159]	INFO	播放视频功能已经设置成：Disable！


In [7]:
from functools import partial
feature_name = 'avgpool'
with open(f'features/{model_name}_features.csv', 'w') as outfile:
    hook = partial(print_feature_hook, fp=outfile)
    find_num = reg_hook_on_module(feature_name, model, hook)
    results = extract(samples, model, transformer, device, fp=outfile)

## 读取数据

In [8]:
#### 特征读取
from onekey_algo.custom.Manager import onekey_show
onekey_show('深度学习特征提取|特征读取')

[2025-12-03 16:35:09 - <frozen onekey_algo.custom.Manager>: 159]	INFO	播放视频功能已经设置成：Disable！


In [9]:
import pandas as pd
features = pd.read_csv(f'features/{model_name}_features.csv', header=None)
features.columns=['ID'] + [f"DL_{i}" for i in range(features.shape[1] - 1)]
features.to_csv(f'features/{model_name}_features.csv', index=False)
features

,ID,DL_0,DL_1,DL_2,DL_3,DL_4,DL_5,DL_6,DL_7,DL_8,...,DL_2038,DL_2039,DL_2040,DL_2041,DL_2042,DL_2043,DL_2044,DL_2045,DL_2046,DL_2047
0,1.nii.png,0.276,1.144,0.177,0.843,0.412,0.439,1.324,0.874,0.349,...,1.210,0.069,0.251,0.845,0.589,0.061,1.077,0.563,0.980,0.773
1,100.nii.png,0.264,0.366,0.353,0.965,0.354,0.107,2.350,0.466,0.921,...,0.438,0.626,1.982,0.697,1.078,0.103,0.891,0.264,1.007,0.483
2,1001.nii.png,0.506,0.621,0.518,1.303,0.355,0.814,1.089,1.240,0.330,...,0.572,0.355,0.681,0.313,0.608,0.052,1.047,0.414,0.732,0.592
3,1003.nii.png,0.396,0.370,0.274,0.216,1.067,0.277,0.757,1.178,0.258,...,0.440,0.448,0.110,0.735,0.584,0.078,0.428,0.397,1.430,0.084
4,101.nii.png,0.605,0.982,0.851,0.929,1.112,0.220,2.031,0.607,0.087,...,0.109,0.154,0.113,0.893,0.489,0.305,0.637,0.198,0.376,0.481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,978.nii.png,0.382,1.180,0.947,0.732,1.602,2.115,0.299,1.516,0.265,...,0.219,0.175,0.389,0.565,0.345,0.117,0.606,0.265,0.388,0.054
756,980.nii.png,0.169,0.322,1.623,0.297,1.701,0.505,1.056,0.522,0.349,...,0.157,0.287,1.195,0.692,0.931,0.126,0.377,0.917,1.500,0.095
757,981.nii.png,0.122,0.840,1.109,0.390,1.003,0.854,1.330,0.806,0.156,...,0.147,0.659,0.454,0.865,1.995,0.139,1.335,0.283,1.388,0.039
758,984.nii.png,0.037,0.332,0.440,0.826,1.736,0.580,1.338,0.017,0.684,...,0.280,0.755,0.497,0.257,1.900,0.051,0.450,0.102,0.515,0.403


### 深度特征压缩

深度学习特征压缩，注意压缩到的维度需要小于样本数

```python
def compress_df_feature(features: pd.DataFrame, dim: int, not_compress: Union[str, List[str]] = None,
                        prefix='') -> pd.DataFrame:
    """
    压缩深度学习特征
    Args:
        features: 特征DataFrame
        dim: 需要压缩到的维度，此值需要小于样本数
        not_compress: 不进行压缩的列。
        prefix: 所有特征的前缀。

    Returns:

    """
```

In [10]:
from onekey_algo.custom.components.comp1 import compress_df_feature

cm_features = compress_df_feature(features=features, dim=8, prefix='DL_', not_compress='ID')
cm_features.to_csv(f'features/{model_name}_compress_features.csv', header=True, index=False)

### 迁移学习

使用Onekey，提取基于迁移学习的模型特征。

In [11]:
#### 特征读取
from onekey_algo.custom.Manager import onekey_show
onekey_show('深度学习特征提取|Onekey迁移学习')

[2025-12-03 16:35:10 - <frozen onekey_algo.custom.Manager>: 159]	INFO	播放视频功能已经设置成：Disable！
